In [287]:
import sys
import os
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import re


In [288]:
import pandas as pd
import numpy as np
import pickle
import csv
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/records_4000.tsv', sep='\t', index_col = 1)
df['genre'] = df['genre'].str[:-1]
labels = list(set([label.strip() for label in df['genre']]))
df=df.fillna(0)
df=df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
# Format of genre is "Rap\n" with the return character
def create_train_validation_test(df, genre):
    # 4000 songs from genre
    labels = list(set([label.strip() for label in df['genre']]))
    otherlabels=list(set(labels)-set([genre]))
     #df['genre'] = df['genre'].str[:].replace([genre], '1')
    #df['genre'] = df['genre'].str[:].replace(otherlabels, '0')
    df_poprock = df.loc[df['genre'] == genre]

    # 4000 songs from not genre
    df_non_poprock = df.loc[df['genre'] != genre].sample(n=4000)

    # Divide pop rock into train and test
    train_poprock, test_poprock = train_test_split(df_poprock, test_size=0.2)

    # Divide non pop rock into train and test
    train_non_poprock, test_non_poprock = train_test_split(df_non_poprock, test_size = 0.2)

    # Combine pop-rock and non-poprock into train/test
    train = train_poprock.append(train_non_poprock)
    test = test_poprock.append(test_non_poprock)

    train, validation = train_test_split(train, test_size=0.2)

    # training, validation, and test data

    X_train = train.drop(['genre'], axis = 1)
    y_train = train[['genre']].replace([genre], '1').replace(otherlabels, '-1')

    X_validation = validation.drop(['genre'], axis = 1)
    y_validation = validation[['genre']].replace([genre], '1').replace(otherlabels, '-1')

    X_test = test.drop(['genre'], axis = 1)
    y_test = test[['genre']].replace([genre], '1').replace(otherlabels, '-1')

    return X_train, y_train, X_validation, y_validation, X_test, y_test

In [644]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,2,1),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(2,5,2),
            nn.ReLU(),
            nn.MaxPool2d(2))

        self.fc = nn.Linear(15, 2)

    def forward(self, x):
        #print "foward 0"
        out = self.layer1(x)
        #print "1"
        out = self.layer2(out)
        #print "2"
        out = out.view(out.size(0), -1)
        #print "out"
        out = self.fc(out)
        #print out
        return out


cnn=CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [633]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,2,2),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(2,3,2),
            nn.ReLU(),
            nn.MaxPool2d(2))
        #self.softmax=nn.Softmax()
        self.fc = nn.Linear(6, 2)

    def forward(self, x):
        #print "foward 0"
        out = self.layer1(x)
        #print "1"
        out = self.layer2(out)
        #print "2"
        out = out.view(out.size(0), -1)
        #out=self.softmax(out)
        #print "out"
        out = self.fc(out)
        #print out
        return out


cnn=CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [622]:
def convertData(X,y):
    myset=[]
    for i in range(0,len(y.values)):
        myX=list(X.values[i])
        myY=max(0,int(y.values[i][0]))
        myset+=[(myX,myY)]
    return myset

    

In [652]:
num_epochs=3
batch_size=5
learning_rate=0.001
X_train, y_train, X_validation, y_validation, X_test, y_test=create_train_validation_test(df,'Country')

train_set=convertData(X_train,y_train)
test_set=convertData(X_validation,y_validation)
random.shuffle(train_set)
random.shuffle(test_set)

In [624]:
# Train the Model
def train(train_set):
    i=0
    for epoch in range(0,num_epochs):
        for i in range(0,len(train_set)-batch_size,batch_size):
            features_list=[]
            lab_list=[]
            for trainexample in train_set[i:i+batch_size]:
                i+=1
                features,lab=trainexample
                features_list+=[[features]]
                lab_list+=[lab]
            features_var = Variable(torch.Tensor(features_list))
            features_var = features_var.view(5,1,4,7)
            label = Variable(torch.LongTensor(lab_list))
            optimizer.zero_grad()
            outputs = cnn(features_var)
            loss = criterion(outputs, label)
            #print i,"loss",loss
            loss.backward()
            optimizer.step()     
            if (i+1) % 5 == 1:
                print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                       %(epoch+1, num_epochs, i%(len(train_set)), (len(train_set))//batch_size, loss.data[0]))


In [625]:

def test(train_set,test_set):
    cnn.eval()

    correct = 0
    total = 0
    labels=[]
    for i in range(0,len(train_set),batch_size):
        features_list=[]
        lab_list=[]
        for features,lab in train_set[i:i+batch_size]:
            features_list+=[[features]]
            lab_list+=[lab]
        features_var = Variable(torch.Tensor(features_list))
        features_var = features_var.view(5,1,4,7)
        outputs = cnn(features_var)
        _, predicted = torch.max(outputs.data, 1)
        predicted=predicted.squeeze(1)
        label = torch.LongTensor(lab_list)
        total += label.size(0)
        correct += (predicted == label).sum()
        labels+=lab_list

    print('Train Accuracy of the model on the test set: %d %%' % (100 * correct / total))
    trainaccu=100 * correct*1.0 / total
    correct = 0
    total = 0
    labels=[]
    for i in range(0,len(test_set),batch_size):
        features_list=[]
        lab_list=[]
        for features,lab in test_set[i:i+batch_size]:
            features_list+=[[features]]
            lab_list+=[lab]
        features_var = Variable(torch.Tensor(features_list))
        features_var = features_var.view(5,1,4,7)
        outputs = cnn(features_var)
        _, predicted = torch.max(outputs.data, 1)
        predicted=predicted.squeeze(1)
        label = torch.LongTensor(lab_list)
        total += label.size(0)
        correct += (predicted == label).sum()
        labels+=lab_list

    print('Test Accuracy of the model on the test set: %d %%' % (100 * correct / total))
    testaccu=100 * correct*1.0 / total
    return trainaccu,testaccu


In [654]:
learning_rate=0.005
cnn=CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
train(train_set)
trainaccu,testaccu=test(train_set,test_set)

Epoch [1/3], Iter [5/1024] Loss: 0.6931
Epoch [1/3], Iter [10/1024] Loss: 0.6768
Epoch [1/3], Iter [15/1024] Loss: 0.6812
Epoch [1/3], Iter [20/1024] Loss: 0.8594
Epoch [1/3], Iter [25/1024] Loss: 0.5719
Epoch [1/3], Iter [30/1024] Loss: 0.7757
Epoch [1/3], Iter [35/1024] Loss: 0.5902
Epoch [1/3], Iter [40/1024] Loss: 0.7426
Epoch [1/3], Iter [45/1024] Loss: 0.6243
Epoch [1/3], Iter [50/1024] Loss: 0.6357
Epoch [1/3], Iter [55/1024] Loss: 0.7805
Epoch [1/3], Iter [60/1024] Loss: 0.7959
Epoch [1/3], Iter [65/1024] Loss: 0.5664
Epoch [1/3], Iter [70/1024] Loss: 0.6630
Epoch [1/3], Iter [75/1024] Loss: 0.8802
Epoch [1/3], Iter [80/1024] Loss: 0.5862
Epoch [1/3], Iter [85/1024] Loss: 0.5734
Epoch [1/3], Iter [90/1024] Loss: 0.6961
Epoch [1/3], Iter [95/1024] Loss: 0.7785
Epoch [1/3], Iter [100/1024] Loss: 0.7587
Epoch [1/3], Iter [105/1024] Loss: 0.7494
Epoch [1/3], Iter [110/1024] Loss: 0.7659
Epoch [1/3], Iter [115/1024] Loss: 0.7001
Epoch [1/3], Iter [120/1024] Loss: 0.7320
Epoch [1/3],

Epoch [1/3], Iter [1490/1024] Loss: 0.7032
Epoch [1/3], Iter [1495/1024] Loss: 0.6995
Epoch [1/3], Iter [1500/1024] Loss: 0.7110
Epoch [1/3], Iter [1505/1024] Loss: 0.6704
Epoch [1/3], Iter [1510/1024] Loss: 0.6928
Epoch [1/3], Iter [1515/1024] Loss: 0.6680
Epoch [1/3], Iter [1520/1024] Loss: 0.6483
Epoch [1/3], Iter [1525/1024] Loss: 0.7134
Epoch [1/3], Iter [1530/1024] Loss: 0.6488
Epoch [1/3], Iter [1535/1024] Loss: 0.7692
Epoch [1/3], Iter [1540/1024] Loss: 0.7531
Epoch [1/3], Iter [1545/1024] Loss: 0.6903
Epoch [1/3], Iter [1550/1024] Loss: 0.6662
Epoch [1/3], Iter [1555/1024] Loss: 0.6327
Epoch [1/3], Iter [1560/1024] Loss: 0.6287
Epoch [1/3], Iter [1565/1024] Loss: 0.7656
Epoch [1/3], Iter [1570/1024] Loss: 0.7533
Epoch [1/3], Iter [1575/1024] Loss: 0.6534
Epoch [1/3], Iter [1580/1024] Loss: 0.7532
Epoch [1/3], Iter [1585/1024] Loss: 0.7525
Epoch [1/3], Iter [1590/1024] Loss: 0.7546
Epoch [1/3], Iter [1595/1024] Loss: 0.6723
Epoch [1/3], Iter [1600/1024] Loss: 0.6541
Epoch [1/3]

Epoch [1/3], Iter [2915/1024] Loss: 0.6041
Epoch [1/3], Iter [2920/1024] Loss: 0.7157
Epoch [1/3], Iter [2925/1024] Loss: 0.6301
Epoch [1/3], Iter [2930/1024] Loss: 0.6772
Epoch [1/3], Iter [2935/1024] Loss: 0.6614
Epoch [1/3], Iter [2940/1024] Loss: 0.6965
Epoch [1/3], Iter [2945/1024] Loss: 0.6381
Epoch [1/3], Iter [2950/1024] Loss: 0.6792
Epoch [1/3], Iter [2955/1024] Loss: 0.6903
Epoch [1/3], Iter [2960/1024] Loss: 0.7379
Epoch [1/3], Iter [2965/1024] Loss: 0.6859
Epoch [1/3], Iter [2970/1024] Loss: 0.7407
Epoch [1/3], Iter [2975/1024] Loss: 0.7349
Epoch [1/3], Iter [2980/1024] Loss: 0.5420
Epoch [1/3], Iter [2985/1024] Loss: 0.6749
Epoch [1/3], Iter [2990/1024] Loss: 0.6742
Epoch [1/3], Iter [2995/1024] Loss: 0.6147
Epoch [1/3], Iter [3000/1024] Loss: 0.5935
Epoch [1/3], Iter [3005/1024] Loss: 0.7490
Epoch [1/3], Iter [3010/1024] Loss: 0.6447
Epoch [1/3], Iter [3015/1024] Loss: 0.6782
Epoch [1/3], Iter [3020/1024] Loss: 0.6121
Epoch [1/3], Iter [3025/1024] Loss: 0.6318
Epoch [1/3]

Epoch [1/3], Iter [4440/1024] Loss: 0.7162
Epoch [1/3], Iter [4445/1024] Loss: 0.7297
Epoch [1/3], Iter [4450/1024] Loss: 0.5261
Epoch [1/3], Iter [4455/1024] Loss: 0.5654
Epoch [1/3], Iter [4460/1024] Loss: 0.5170
Epoch [1/3], Iter [4465/1024] Loss: 0.7643
Epoch [1/3], Iter [4470/1024] Loss: 0.6687
Epoch [1/3], Iter [4475/1024] Loss: 0.7155
Epoch [1/3], Iter [4480/1024] Loss: 0.6889
Epoch [1/3], Iter [4485/1024] Loss: 0.5912
Epoch [1/3], Iter [4490/1024] Loss: 0.6858
Epoch [1/3], Iter [4495/1024] Loss: 0.7816
Epoch [1/3], Iter [4500/1024] Loss: 0.7564
Epoch [1/3], Iter [4505/1024] Loss: 0.6470
Epoch [1/3], Iter [4510/1024] Loss: 0.6519
Epoch [1/3], Iter [4515/1024] Loss: 0.6763
Epoch [1/3], Iter [4520/1024] Loss: 0.4929
Epoch [1/3], Iter [4525/1024] Loss: 0.6273
Epoch [1/3], Iter [4530/1024] Loss: 0.7215
Epoch [1/3], Iter [4535/1024] Loss: 0.7032
Epoch [1/3], Iter [4540/1024] Loss: 0.5272
Epoch [1/3], Iter [4545/1024] Loss: 0.6608
Epoch [1/3], Iter [4550/1024] Loss: 0.6829
Epoch [1/3]

Epoch [2/3], Iter [745/1024] Loss: 0.8274
Epoch [2/3], Iter [750/1024] Loss: 0.7073
Epoch [2/3], Iter [755/1024] Loss: 0.8046
Epoch [2/3], Iter [760/1024] Loss: 0.4361
Epoch [2/3], Iter [765/1024] Loss: 0.6124
Epoch [2/3], Iter [770/1024] Loss: 0.7941
Epoch [2/3], Iter [775/1024] Loss: 0.7567
Epoch [2/3], Iter [780/1024] Loss: 0.7010
Epoch [2/3], Iter [785/1024] Loss: 0.9218
Epoch [2/3], Iter [790/1024] Loss: 0.6602
Epoch [2/3], Iter [795/1024] Loss: 0.5396
Epoch [2/3], Iter [800/1024] Loss: 0.6600
Epoch [2/3], Iter [805/1024] Loss: 0.6954
Epoch [2/3], Iter [810/1024] Loss: 0.6433
Epoch [2/3], Iter [815/1024] Loss: 0.6125
Epoch [2/3], Iter [820/1024] Loss: 0.6902
Epoch [2/3], Iter [825/1024] Loss: 0.6732
Epoch [2/3], Iter [830/1024] Loss: 0.5003
Epoch [2/3], Iter [835/1024] Loss: 0.6445
Epoch [2/3], Iter [840/1024] Loss: 0.4508
Epoch [2/3], Iter [845/1024] Loss: 0.7624
Epoch [2/3], Iter [850/1024] Loss: 0.6010
Epoch [2/3], Iter [855/1024] Loss: 0.6465
Epoch [2/3], Iter [860/1024] Loss:

...

Epoch [2/3], Iter [4600/1024] Loss: 0.4613
Epoch [2/3], Iter [4605/1024] Loss: 0.7792
Epoch [2/3], Iter [4610/1024] Loss: 0.9049
Epoch [2/3], Iter [4615/1024] Loss: 0.6892
Epoch [2/3], Iter [4620/1024] Loss: 0.6153
Epoch [2/3], Iter [4625/1024] Loss: 0.7805
Epoch [2/3], Iter [4630/1024] Loss: 0.5086
Epoch [2/3], Iter [4635/1024] Loss: 0.4927
Epoch [2/3], Iter [4640/1024] Loss: 0.8616
Epoch [2/3], Iter [4645/1024] Loss: 0.5624
Epoch [2/3], Iter [4650/1024] Loss: 0.5862
Epoch [2/3], Iter [4655/1024] Loss: 0.6616
Epoch [2/3], Iter [4660/1024] Loss: 0.4172
Epoch [2/3], Iter [4665/1024] Loss: 0.4697
Epoch [2/3], Iter [4670/1024] Loss: 0.4272
Epoch [2/3], Iter [4675/1024] Loss: 0.7061
Epoch [2/3], Iter [4680/1024] Loss: 1.0451
Epoch [2/3], Iter [4685/1024] Loss: 0.7098
Epoch [2/3], Iter [4690/1024] Loss: 0.7948
Epoch [2/3], Iter [4695/1024] Loss: 0.8083
Epoch [2/3], Iter [4700/1024] Loss: 0.6098
Epoch [2/3], Iter [4705/1024] Loss: 0.5868
Epoch [2/3], Iter [4710/1024] Loss: 0.6692
Epoch [2/3]

Epoch [3/3], Iter [900/1024] Loss: 0.6179
Epoch [3/3], Iter [905/1024] Loss: 0.3898
Epoch [3/3], Iter [910/1024] Loss: 0.5155
Epoch [3/3], Iter [915/1024] Loss: 0.5608
Epoch [3/3], Iter [920/1024] Loss: 0.5985
Epoch [3/3], Iter [925/1024] Loss: 0.7497
Epoch [3/3], Iter [930/1024] Loss: 0.6487
Epoch [3/3], Iter [935/1024] Loss: 0.4989
Epoch [3/3], Iter [940/1024] Loss: 1.1098
Epoch [3/3], Iter [945/1024] Loss: 0.7371
Epoch [3/3], Iter [950/1024] Loss: 0.7495
Epoch [3/3], Iter [955/1024] Loss: 0.8617
Epoch [3/3], Iter [960/1024] Loss: 0.7699
Epoch [3/3], Iter [965/1024] Loss: 0.5316
Epoch [3/3], Iter [970/1024] Loss: 0.9697
Epoch [3/3], Iter [975/1024] Loss: 0.6326
Epoch [3/3], Iter [980/1024] Loss: 0.8228
Epoch [3/3], Iter [985/1024] Loss: 0.7398
Epoch [3/3], Iter [990/1024] Loss: 0.8021
Epoch [3/3], Iter [995/1024] Loss: 0.5176
Epoch [3/3], Iter [1000/1024] Loss: 0.7780
Epoch [3/3], Iter [1005/1024] Loss: 0.4835
Epoch [3/3], Iter [1010/1024] Loss: 0.4689
Epoch [3/3], Iter [1015/1024] L

Epoch [3/3], Iter [2410/1024] Loss: 0.6267
Epoch [3/3], Iter [2415/1024] Loss: 0.6704
Epoch [3/3], Iter [2420/1024] Loss: 0.4599
Epoch [3/3], Iter [2425/1024] Loss: 0.4885
Epoch [3/3], Iter [2430/1024] Loss: 0.5661
Epoch [3/3], Iter [2435/1024] Loss: 0.7037
Epoch [3/3], Iter [2440/1024] Loss: 0.5053
Epoch [3/3], Iter [2445/1024] Loss: 0.3485
Epoch [3/3], Iter [2450/1024] Loss: 0.5922
Epoch [3/3], Iter [2455/1024] Loss: 0.7033
Epoch [3/3], Iter [2460/1024] Loss: 0.6904
Epoch [3/3], Iter [2465/1024] Loss: 0.5067
Epoch [3/3], Iter [2470/1024] Loss: 0.7571
Epoch [3/3], Iter [2475/1024] Loss: 0.7775
Epoch [3/3], Iter [2480/1024] Loss: 0.6273
Epoch [3/3], Iter [2485/1024] Loss: 0.4914
Epoch [3/3], Iter [2490/1024] Loss: 0.4257
Epoch [3/3], Iter [2495/1024] Loss: 0.7391
Epoch [3/3], Iter [2500/1024] Loss: 0.4312
Epoch [3/3], Iter [2505/1024] Loss: 0.7856
Epoch [3/3], Iter [2510/1024] Loss: 0.7661
Epoch [3/3], Iter [2515/1024] Loss: 0.4369
Epoch [3/3], Iter [2520/1024] Loss: 0.8548
Epoch [3/3]

Epoch [3/3], Iter [3940/1024] Loss: 0.9264
Epoch [3/3], Iter [3945/1024] Loss: 0.4978
Epoch [3/3], Iter [3950/1024] Loss: 0.9614
Epoch [3/3], Iter [3955/1024] Loss: 0.5481
Epoch [3/3], Iter [3960/1024] Loss: 0.5325
Epoch [3/3], Iter [3965/1024] Loss: 0.6266
Epoch [3/3], Iter [3970/1024] Loss: 0.5962
Epoch [3/3], Iter [3975/1024] Loss: 0.4516
Epoch [3/3], Iter [3980/1024] Loss: 0.7414
Epoch [3/3], Iter [3985/1024] Loss: 0.5167
Epoch [3/3], Iter [3990/1024] Loss: 0.8424
Epoch [3/3], Iter [3995/1024] Loss: 0.6711
Epoch [3/3], Iter [4000/1024] Loss: 0.6446
Epoch [3/3], Iter [4005/1024] Loss: 0.5183
Epoch [3/3], Iter [4010/1024] Loss: 0.5505
Epoch [3/3], Iter [4015/1024] Loss: 0.6141
Epoch [3/3], Iter [4020/1024] Loss: 0.5020
Epoch [3/3], Iter [4025/1024] Loss: 0.7343
Epoch [3/3], Iter [4030/1024] Loss: 0.5007
Epoch [3/3], Iter [4035/1024] Loss: 0.4568
Epoch [3/3], Iter [4040/1024] Loss: 0.5389
Epoch [3/3], Iter [4045/1024] Loss: 0.9120
Epoch [3/3], Iter [4050/1024] Loss: 0.8146
Epoch [3/3]

Train Accuracy of the model on the test set: 66 %
Test Accuracy of the model on the test set: 66 %


In [647]:
accu=[]
for labelname in labels:
    X_train, y_train, X_validation, y_validation, X_test, y_test=create_train_validation_test(df,labelname)
    train_set=convertData(X_train,y_train)
    test_set=convertData(X_validation,y_validation)
    random.shuffle(train_set)
    random.shuffle(test_set)
    cnn=CNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    train(train_set)
    trainaccu,testaccu=test(train_set,test_set)
    accu+=[(labelname,trainaccu,testaccu)]


Epoch [1/3], Iter [5/1024] Loss: 3.8750
Epoch [1/3], Iter [10/1024] Loss: 2.5392
Epoch [1/3], Iter [15/1024] Loss: 1.8925
Epoch [1/3], Iter [20/1024] Loss: 1.0247
Epoch [1/3], Iter [25/1024] Loss: 1.2152
Epoch [1/3], Iter [30/1024] Loss: 2.4510
Epoch [1/3], Iter [35/1024] Loss: 2.2093
Epoch [1/3], Iter [40/1024] Loss: 1.7768
Epoch [1/3], Iter [45/1024] Loss: 1.2159
Epoch [1/3], Iter [50/1024] Loss: 0.0377
Epoch [1/3], Iter [55/1024] Loss: 0.9178
Epoch [1/3], Iter [60/1024] Loss: 2.1000
Epoch [1/3], Iter [65/1024] Loss: 1.2193
Epoch [1/3], Iter [70/1024] Loss: 2.0371
Epoch [1/3], Iter [75/1024] Loss: 1.6671
Epoch [1/3], Iter [80/1024] Loss: 1.0760
Epoch [1/3], Iter [85/1024] Loss: 1.2373
Epoch [1/3], Iter [90/1024] Loss: 2.4395
Epoch [1/3], Iter [95/1024] Loss: 0.8880
Epoch [1/3], Iter [100/1024] Loss: 0.9805
Epoch [1/3], Iter [105/1024] Loss: 0.8816
Epoch [1/3], Iter [110/1024] Loss: 1.1943
Epoch [1/3], Iter [115/1024] Loss: 1.5598
Epoch [1/3], Iter [120/1024] Loss: 1.0592
Epoch [1/3],

Epoch [1/3], Iter [1440/1024] Loss: 0.5538
Epoch [1/3], Iter [1445/1024] Loss: 0.6246
Epoch [1/3], Iter [1450/1024] Loss: 0.5407
Epoch [1/3], Iter [1455/1024] Loss: 0.6855
Epoch [1/3], Iter [1460/1024] Loss: 0.6398
Epoch [1/3], Iter [1465/1024] Loss: 0.6140
Epoch [1/3], Iter [1470/1024] Loss: 0.4419
Epoch [1/3], Iter [1475/1024] Loss: 0.6246
Epoch [1/3], Iter [1480/1024] Loss: 0.5197
Epoch [1/3], Iter [1485/1024] Loss: 0.6765
Epoch [1/3], Iter [1490/1024] Loss: 0.4105
Epoch [1/3], Iter [1495/1024] Loss: 0.6109
Epoch [1/3], Iter [1500/1024] Loss: 0.7266
Epoch [1/3], Iter [1505/1024] Loss: 0.4237
Epoch [1/3], Iter [1510/1024] Loss: 0.8207
Epoch [1/3], Iter [1515/1024] Loss: 0.7014
Epoch [1/3], Iter [1520/1024] Loss: 0.5256
Epoch [1/3], Iter [1525/1024] Loss: 0.7184
Epoch [1/3], Iter [1530/1024] Loss: 0.5373
Epoch [1/3], Iter [1535/1024] Loss: 0.4665
Epoch [1/3], Iter [1540/1024] Loss: 0.4386
Epoch [1/3], Iter [1545/1024] Loss: 0.5899
Epoch [1/3], Iter [1550/1024] Loss: 0.5287
Epoch [1/3]

Epoch [1/3], Iter [2965/1024] Loss: 0.5535
Epoch [1/3], Iter [2970/1024] Loss: 0.6377
Epoch [1/3], Iter [2975/1024] Loss: 0.3708
Epoch [1/3], Iter [2980/1024] Loss: 0.6877
Epoch [1/3], Iter [2985/1024] Loss: 0.6989
Epoch [1/3], Iter [2990/1024] Loss: 0.4962
Epoch [1/3], Iter [2995/1024] Loss: 0.7451
Epoch [1/3], Iter [3000/1024] Loss: 0.4794
Epoch [1/3], Iter [3005/1024] Loss: 0.4778
Epoch [1/3], Iter [3010/1024] Loss: 0.6153
Epoch [1/3], Iter [3015/1024] Loss: 0.5736
Epoch [1/3], Iter [3020/1024] Loss: 0.6000
Epoch [1/3], Iter [3025/1024] Loss: 0.8477
Epoch [1/3], Iter [3030/1024] Loss: 0.6716
Epoch [1/3], Iter [3035/1024] Loss: 0.5909
Epoch [1/3], Iter [3040/1024] Loss: 0.6562
Epoch [1/3], Iter [3045/1024] Loss: 0.4073
Epoch [1/3], Iter [3050/1024] Loss: 0.8977
Epoch [1/3], Iter [3055/1024] Loss: 0.7271
Epoch [1/3], Iter [3060/1024] Loss: 0.6323
Epoch [1/3], Iter [3065/1024] Loss: 0.5274
Epoch [1/3], Iter [3070/1024] Loss: 0.7320
Epoch [1/3], Iter [3075/1024] Loss: 0.8635
Epoch [1/3]

Epoch [1/3], Iter [4430/1024] Loss: 0.5891
Epoch [1/3], Iter [4435/1024] Loss: 0.5011
Epoch [1/3], Iter [4440/1024] Loss: 0.3774
Epoch [1/3], Iter [4445/1024] Loss: 0.3303
Epoch [1/3], Iter [4450/1024] Loss: 0.7057
Epoch [1/3], Iter [4455/1024] Loss: 0.3670
Epoch [1/3], Iter [4460/1024] Loss: 0.6294
Epoch [1/3], Iter [4465/1024] Loss: 0.6087
Epoch [1/3], Iter [4470/1024] Loss: 0.6134
Epoch [1/3], Iter [4475/1024] Loss: 0.7030
Epoch [1/3], Iter [4480/1024] Loss: 0.8117
Epoch [1/3], Iter [4485/1024] Loss: 0.4611
Epoch [1/3], Iter [4490/1024] Loss: 0.4745
Epoch [1/3], Iter [4495/1024] Loss: 0.6481
Epoch [1/3], Iter [4500/1024] Loss: 0.5059
Epoch [1/3], Iter [4505/1024] Loss: 0.4467
Epoch [1/3], Iter [4510/1024] Loss: 0.7114
Epoch [1/3], Iter [4515/1024] Loss: 0.4241
Epoch [1/3], Iter [4520/1024] Loss: 0.4577
Epoch [1/3], Iter [4525/1024] Loss: 0.7678
Epoch [1/3], Iter [4530/1024] Loss: 0.5487
Epoch [1/3], Iter [4535/1024] Loss: 0.6216
Epoch [1/3], Iter [4540/1024] Loss: 0.3844
Epoch [1/3]

Epoch [2/3], Iter [590/1024] Loss: 0.6197
Epoch [2/3], Iter [595/1024] Loss: 0.6657
Epoch [2/3], Iter [600/1024] Loss: 0.5307
Epoch [2/3], Iter [605/1024] Loss: 0.6312
Epoch [2/3], Iter [610/1024] Loss: 0.6481
Epoch [2/3], Iter [615/1024] Loss: 0.6004
Epoch [2/3], Iter [620/1024] Loss: 0.6233
Epoch [2/3], Iter [625/1024] Loss: 0.5778
Epoch [2/3], Iter [630/1024] Loss: 0.4853
Epoch [2/3], Iter [635/1024] Loss: 0.3744
Epoch [2/3], Iter [640/1024] Loss: 0.5928
Epoch [2/3], Iter [645/1024] Loss: 0.5505
Epoch [2/3], Iter [650/1024] Loss: 0.4746
Epoch [2/3], Iter [655/1024] Loss: 0.4337
Epoch [2/3], Iter [660/1024] Loss: 0.3727
Epoch [2/3], Iter [665/1024] Loss: 0.5684
Epoch [2/3], Iter [670/1024] Loss: 0.3673
Epoch [2/3], Iter [675/1024] Loss: 0.5682
Epoch [2/3], Iter [680/1024] Loss: 0.4903
Epoch [2/3], Iter [685/1024] Loss: 0.5431
Epoch [2/3], Iter [690/1024] Loss: 0.6097
Epoch [2/3], Iter [695/1024] Loss: 0.6892
Epoch [2/3], Iter [700/1024] Loss: 0.5808
Epoch [2/3], Iter [705/1024] Loss:

...

Epoch [2/3], Iter [4225/1024] Loss: 0.9843
Epoch [2/3], Iter [4230/1024] Loss: 1.0999
Epoch [2/3], Iter [4235/1024] Loss: 0.7866
Epoch [2/3], Iter [4240/1024] Loss: 0.6459
Epoch [2/3], Iter [4245/1024] Loss: 0.5469
Epoch [2/3], Iter [4250/1024] Loss: 0.6395
Epoch [2/3], Iter [4255/1024] Loss: 0.6326
Epoch [2/3], Iter [4260/1024] Loss: 0.7172
Epoch [2/3], Iter [4265/1024] Loss: 0.6911
Epoch [2/3], Iter [4270/1024] Loss: 0.8176
Epoch [2/3], Iter [4275/1024] Loss: 0.7741
Epoch [2/3], Iter [4280/1024] Loss: 0.6697
Epoch [2/3], Iter [4285/1024] Loss: 0.5536
Epoch [2/3], Iter [4290/1024] Loss: 0.6706
Epoch [2/3], Iter [4295/1024] Loss: 0.7024
Epoch [2/3], Iter [4300/1024] Loss: 0.5974
Epoch [2/3], Iter [4305/1024] Loss: 0.7619
Epoch [2/3], Iter [4310/1024] Loss: 0.6571
Epoch [2/3], Iter [4315/1024] Loss: 0.6319
Epoch [2/3], Iter [4320/1024] Loss: 0.7249
Epoch [2/3], Iter [4325/1024] Loss: 1.0555
Epoch [2/3], Iter [4330/1024] Loss: 0.6306
Epoch [2/3], Iter [4335/1024] Loss: 0.7945
Epoch [2/3]

Epoch [3/3], Iter [540/1024] Loss: 0.6885
Epoch [3/3], Iter [545/1024] Loss: 0.7227
Epoch [3/3], Iter [550/1024] Loss: 0.6829
Epoch [3/3], Iter [555/1024] Loss: 0.6638
Epoch [3/3], Iter [560/1024] Loss: 0.7518
Epoch [3/3], Iter [565/1024] Loss: 0.6227
Epoch [3/3], Iter [570/1024] Loss: 0.4898
Epoch [3/3], Iter [575/1024] Loss: 0.7971
Epoch [3/3], Iter [580/1024] Loss: 0.7146
Epoch [3/3], Iter [585/1024] Loss: 0.5475
Epoch [3/3], Iter [590/1024] Loss: 0.7238
Epoch [3/3], Iter [595/1024] Loss: 0.6565
Epoch [3/3], Iter [600/1024] Loss: 0.8152
Epoch [3/3], Iter [605/1024] Loss: 0.6137
Epoch [3/3], Iter [610/1024] Loss: 0.8991
Epoch [3/3], Iter [615/1024] Loss: 0.4604
Epoch [3/3], Iter [620/1024] Loss: 0.8045
Epoch [3/3], Iter [625/1024] Loss: 0.7579
Epoch [3/3], Iter [630/1024] Loss: 0.7493
Epoch [3/3], Iter [635/1024] Loss: 0.8268
Epoch [3/3], Iter [640/1024] Loss: 0.7991
Epoch [3/3], Iter [645/1024] Loss: 0.8201
Epoch [3/3], Iter [650/1024] Loss: 0.5920
Epoch [3/3], Iter [655/1024] Loss:

Epoch [3/3], Iter [1980/1024] Loss: 0.6416
Epoch [3/3], Iter [1985/1024] Loss: 0.7680
Epoch [3/3], Iter [1990/1024] Loss: 0.5028
Epoch [3/3], Iter [1995/1024] Loss: 0.5975
Epoch [3/3], Iter [2000/1024] Loss: 0.9632
Epoch [3/3], Iter [2005/1024] Loss: 0.6823
Epoch [3/3], Iter [2010/1024] Loss: 0.5714
Epoch [3/3], Iter [2015/1024] Loss: 0.5652
Epoch [3/3], Iter [2020/1024] Loss: 0.4954
Epoch [3/3], Iter [2025/1024] Loss: 0.8819
Epoch [3/3], Iter [2030/1024] Loss: 0.7969
Epoch [3/3], Iter [2035/1024] Loss: 0.3091
Epoch [3/3], Iter [2040/1024] Loss: 1.0558
Epoch [3/3], Iter [2045/1024] Loss: 0.9966
Epoch [3/3], Iter [2050/1024] Loss: 0.8557
Epoch [3/3], Iter [2055/1024] Loss: 0.6246
Epoch [3/3], Iter [2060/1024] Loss: 0.8215
Epoch [3/3], Iter [2065/1024] Loss: 0.6016
Epoch [3/3], Iter [2070/1024] Loss: 0.7097
Epoch [3/3], Iter [2075/1024] Loss: 0.6365
Epoch [3/3], Iter [2080/1024] Loss: 0.8541
Epoch [3/3], Iter [2085/1024] Loss: 0.5578
Epoch [3/3], Iter [2090/1024] Loss: 0.7718
Epoch [3/3]

Epoch [3/3], Iter [3490/1024] Loss: 0.8964
Epoch [3/3], Iter [3495/1024] Loss: 0.5979
Epoch [3/3], Iter [3500/1024] Loss: 0.9213
Epoch [3/3], Iter [3505/1024] Loss: 0.9346
Epoch [3/3], Iter [3510/1024] Loss: 0.8537
Epoch [3/3], Iter [3515/1024] Loss: 0.5593
Epoch [3/3], Iter [3520/1024] Loss: 0.6215
Epoch [3/3], Iter [3525/1024] Loss: 0.8028
Epoch [3/3], Iter [3530/1024] Loss: 0.7799
Epoch [3/3], Iter [3535/1024] Loss: 1.0312
Epoch [3/3], Iter [3540/1024] Loss: 0.4815
Epoch [3/3], Iter [3545/1024] Loss: 0.7874
Epoch [3/3], Iter [3550/1024] Loss: 0.7084
Epoch [3/3], Iter [3555/1024] Loss: 0.5504
Epoch [3/3], Iter [3560/1024] Loss: 0.7095
Epoch [3/3], Iter [3565/1024] Loss: 0.7187
Epoch [3/3], Iter [3570/1024] Loss: 0.6867
Epoch [3/3], Iter [3575/1024] Loss: 0.6859
Epoch [3/3], Iter [3580/1024] Loss: 0.6925
Epoch [3/3], Iter [3585/1024] Loss: 0.6873
Epoch [3/3], Iter [3590/1024] Loss: 0.6695
Epoch [3/3], Iter [3595/1024] Loss: 0.7694
Epoch [3/3], Iter [3600/1024] Loss: 0.7243
Epoch [3/3]

Epoch [3/3], Iter [5045/1024] Loss: 0.6511
Epoch [3/3], Iter [5050/1024] Loss: 0.4116
Epoch [3/3], Iter [5055/1024] Loss: 0.8082
Epoch [3/3], Iter [5060/1024] Loss: 0.5331
Epoch [3/3], Iter [5065/1024] Loss: 0.8763
Epoch [3/3], Iter [5070/1024] Loss: 0.3333
Epoch [3/3], Iter [5075/1024] Loss: 0.7411
Epoch [3/3], Iter [5080/1024] Loss: 0.8530
Epoch [3/3], Iter [5085/1024] Loss: 0.9265
Epoch [3/3], Iter [5090/1024] Loss: 0.3459
Epoch [3/3], Iter [5095/1024] Loss: 0.7238
Epoch [3/3], Iter [5100/1024] Loss: 0.9457
Epoch [3/3], Iter [5105/1024] Loss: 0.5385
Epoch [3/3], Iter [5110/1024] Loss: 0.5170
Epoch [3/3], Iter [5115/1024] Loss: 0.6660
Train Accuracy of the model on the test set: 53 %
Test Accuracy of the model on the test set: 55 %


In [648]:
accu

[('Reggae', 70.21484375, 71.015625),
 ('Pop_Rock', 67.34375, 70.625),
 ('Country', 62.48046875, 59.765625),
 ('Jazz', 62.16796875, 60.390625),
 ('Vocal', 69.53125, 70.3125),
 ('New Age', 50.95703125, 50.15625),
 ('Latin', 60.9765625, 63.203125),
 ('Rap', 71.85546875, 70.390625),
 ('RnB', 54.8046875, 55.78125),
 ('International', 51.66015625, 49.609375),
 ('Blues', 54.70703125, 52.578125),
 ('Electronic', 67.578125, 68.046875),
 ('Folk', 53.2421875, 55.859375)]

In [222]:
trainaccu,testaccu

(50.0, 50.0)

In [655]:
trainaccu,testaccu


(66.81640625, 66.796875)